# Check WebMaps for Broken Layers

This script is designed to check your web maps and report if it has found any broken or unreachable layers. This script posts a status message on a Microsoft Teams channel with a list of the broken layers.

Before running this script, you must create a "Secrets" CSV and place it in ArcGIS Online to hold the API URL for the webhook connecting to Microsoft Teams. Use the <a href="https://github.com/NAPSG/Public-Code-Beta/blob/main/Check%20Webmaps/ReadMe.md">ReadMe document in the Github folder</a> to create and update the CSV.

This script is based on an Esri Sample Notebook for the same purpose but has been modified to post a message on Microsoft Teams instead of sending an email. <a href="https://napsg.maps.arcgis.com/home/notebook/notebook.html?id=f3923d568a1a489594391f7cfb9a3642">The original script can be found here.</a>

In [ ]:
import csv
import smtplib
import requests
import logging
log = logging.getLogger()

from arcgis.gis import GIS
from arcgis.mapping import WebMap

gis = GIS("home")

## Notifications

<b>Change line 1 - </b>Change the SECRET_CSV_ITEM_ID to the itemID of your secrets CSV that holds your webhook URL.

In [ ]:
SECRET_CSV_ITEM_ID = ''

def get_secrets(gis=gis,
                secret_csv_item_id = SECRET_CSV_ITEM_ID):
    """Returns a dict of { secret_key : secret_value } from the 
    secrets.csv item. See the 'Notifications' notebook in the 
    examples gallery for more information.
    """
    try:
        item = gis.content.get(secret_csv_item_id)
        with open(item.download(), 'r') as local_item_file:
            reader = csv.DictReader(local_item_file)
            return { rows['secret_key'] : rows['secret_value'] \
                     for rows in reader }
    except Exception:
        return {}

<b>Change line 2</b> - within the [ ] where it says <i>Teams_Webhook</i>, input the name of your webhook from the Secrets CSV. This will grab the correct webhook URL to be used
(Be sure to keep it in quotes and within the brackets).

In [ ]:
def notify_channel_teams(message):
    webhook_url = get_secrets()['Teams_Webhook']
    response = requests.post(webhook_url, json={"text" : message})
    
    # Check that the response looks OK; warn the user if it doesn't
    if response.ok:
        return True
    else:
        log.warning("Teams POST returned a bad response:")
        log.warning(response.text)
        return Fal

## Configure Behavior

Below are three options for checking the web maps. See the comments above each section for details.

<p style="color:red;"><b>At least one of these must be made True or the script will have an error.</b></p> You can make several options true. For example, if you want to check ever webmap in a few groups and a few specific webmaps, you can make Groups as True, define the groups, and keep the specific maps True and define the webmaps.

In [ ]:
# Set to `True` you would like to check ALL webmaps in your organization.
CHECK_ALL_ITEMS = False

# If `CHECK_THESE_GROUPS` is `True`, the script will check all items in these groups. Un-comment the secoond line here and specify the groups. You can input as many as you want.
CHECK_THESE_GROUPS = False
#CHECK_THESE_GROUPS = ['group_name_1', 'group_name_2']

# This is to specify maps to check. Place the ITEM ID of the webmaps in the quotes, then add the items to the list below.
CHECK_THSES_MAPS = True
item1 = gis.content.get("4972d96596c4497face054ac86a801c0")
item2 = gis.content.get("bf7f6d07e6364d19aadd3f5dbc152cd4")
#item3 = gis.content.get("")
#item4 = gis.content.get("")
#item5 = gis.content.get("")

# Add all open lines above to this list. For example, if you have items 1-3 filled, the list should look like: webmaps = [item1, item2, item3]
webmaps = [item1, item2]

## WebMap Parsing Logic

Checks to see if the layers are reachable.

You will not have to edit any of these. The notes below are for information on what they are doing.

In [ ]:
def is_url_reachable(url):
    """Returns a bool representing if the URL is reachable"""
    try:
        response = requests.get(url)
        return response.ok
    except Exception as e:
        return False

This function will test the URLs of all operational layers and basemap layers of a web map. This function will return a list of reachable and unreachable layers.

In [ ]:
def test_urls_in_webmap(webmap_item):
    """Takes in an `Item` class instance of a Web Map Item.
    Tests if all operational layers and basemap layers are
    reachable. Returns a tuple of (reachable, unreachable), 
    with each tuple entry being a list of layers/basemaps JSON.
    """
    reachable = []
    unreachable = []
    wm = WebMap(webmap_item)

    # Concatanate all operational layers and basemap layers to 1 list
    all_layers = list(wm.layers)
    if hasattr(wm.basemap, 'baseMapLayers'):
        all_layers += wm.basemap.baseMapLayers

    # Test all of the layers, return the results
    for layer in [layer for layer in all_layers \
                  if hasattr(layer, 'url')]:
        if is_url_reachable(layer.url):
            log.debug(f"    [✓] url {layer.url} reachable")
            reachable.append(layer)
        else:
            log.debug(f"    [X] url {layer.url} NOT reachable")
            unreachable.append(layer)
    return (reachable, unreachable)

This function assembles the message to be posted in Teams. This version of the script will only list unreachable layers in the final post.

In [ ]:
def assemble_message(webmap_item, reachable, unreachable):
    """Makes a human readable message with the args passed in"""
    def _assemble_bullet_point(layer, is_reachable):
        """Internal function to assemble one bullet point from a layer"""
        icon = '✅' if is_reachable else '❌'
        return f"* {icon} {layer.title}"

    # Assemble a string representation of both lists
    reachable_str = "\n".join(_assemble_bullet_point(layer, True) \
                               for layer in reachable)
    unreachable_str = "\n".join(_assemble_bullet_point(layer, False) \
                               for layer in unreachable)

    # Asemble the message and return it
    return f"{webmap_item.title} contains unreachable URLs. "\
           f"You can view the webmap here: {webmap_item.homepage}\n"\
           f"\n"\
           f"Unreachable Layers\n"\
           f"-------------------\n"\
           f"{unreachable_str}"

In [ ]:
def handle_unreachable(webmap_item, reachable, unreachable,
                        gis=gis):
    """Called whenever we encounter a WebMap with broken URLs. Will 
    assemble an appropriate message, and send it out to the previously
    configured emails.
    """
    # Assemble the message and send it
    message = assemble_message(webmap_item, reachable, unreachable)
    subject = f"WebMap '{webmap_item.id}' contains broken URLs"
    if notify_channel_teams(message):
        return True
    else:
        log.warning(f"Error emailing users about WebMap {webmap_item.id}.")
        return False

In [ ]:
def get_items_to_check():
    """Generator function that will yield Items depending on how you
    configured your notebook. Will either yield every item in an 
    organization, or will yield items in specific groups.
    """
    if CHECK_ALL_ITEMS:
        for user in gis.users.search():
            for item in user.items(max_items=999999999):
                # For the user's root folder
                yield item
            for folder in user.folders:
                # For all the user's other folders
                for item in user.items(folder, max_items=999999999):
                    yield item
    if CHECK_THESE_GROUPS:
        for group_name in CHECK_THESE_GROUPS:
            group = gis.groups.search(f"title: {group_name}")[0]
            for item in group.content():
                yield item
                
    if CHECK_THSES_MAPS:
        for maps in webmaps:
            yield maps
    else:
        print("An Error occured in the configure behavior")

This main function is what takes all the tools created above and makes them run in the right order.

In [ ]:
def main():
    print("Notebook is now running, please wait...\n-----")
    for item in get_items_to_check():
        print(f"\rChecking item {item.id}", end="")
        if item.type == "Web Map":
            reachable, unreachable = test_urls_in_webmap(item)
            if unreachable:
                print(f"\nWebmap {item.id} unreachable. Notifying...")
                handle_unreachable(item, reachable, unreachable)
    print("-----\nNotebook completed running.")

In [ ]:
main()

Test your script and make sure it has the desired effect. Once you are happy with your script, you may have the ability to schedule the script to run automatically. Go to the top toolbar and click Tasks. Then click Create Task. Name your task and click Next and choose how often you want your script to run. For more information on creating tasks, see the <a href="https://doc.arcgis.com/en/arcgis-online/create-maps/prepare-a-notebook-for-automated-execution.htm">Schedule a notebook Task Esri tech document.</a>